In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV

import seaborn as sns
sns.set(rc={'figure.figsize':(20, 20)})

In [ ]:
train_df = pd.read_csv("../input/students-drop-out-prediction/train.csv")

In [ ]:
train_df.info()

In [ ]:
all_col_names = list(set(train_df.columns) - set(["label", "id"])) 
binary_col_names = ["v_1", "v_26", "v_11", "v_14", "v_30", "v_28", "v_9", "v_27"]
non_binary_col_names = list(set(all_col_names) - set(binary_col_names))

In [ ]:
transformer = ColumnTransformer(
    [
        ("drop_id", "drop", ["id"]),
        ("scale_non_binary", StandardScaler(), non_binary_col_names)
    ],
    remainder="passthrough"
)
transform_train_df = transformer.fit_transform(train_df)
feature_names = [name.split("__")[1] for name in transformer.get_feature_names_out()]
transform_train_df = pd.DataFrame(data=transform_train_df, columns=feature_names)
transform_train_df = transform_train_df.astype(dict([(col, "int") for col in binary_col_names]))

In [ ]:
sns.heatmap(transform_train_df.corr())

In [ ]:
pca = PCA(random_state=1)
reduced_transform_train_df = pca.fit_transform(transform_train_df[non_binary_col_names])

In [ ]:
pca.explained_variance_ratio_[:4]

In [ ]:
reduced_transform_train_df = pd.DataFrame(data = reduced_transform_train_df[:,:4], columns = ["PC1", "PC2", "PC3", "PC4"])
reduced_transform_train_df

In [ ]:
for col in binary_col_names:
    reduced_transform_train_df[col] = transform_train_df[col]

In [ ]:
X_train_com = reduced_transform_train_df
y_train_com = transform_train_df["label"]

In [ ]:
param_grid = {
    "alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000],
    "learning_rate": ["optimal", "invscaling", "adaptive"],
    "eta0": [1, 10, 100],
    "penalty": ["l1", "l2", "elasticnet"]
}
model = SGDClassifier(random_state=1, loss="log", penalty="elasticnet")
search = GridSearchCV(model, param_grid=param_grid, scoring="f1_macro", refit=True, cv=5)
search.fit(X_train_com, y_train_com)

In [ ]:
search.best_params_

In [ ]:
search.best_score_

In [4]:
import numpy as np
np.logspace(-2, 3, 6)

array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03])